<a href="https://colab.research.google.com/github/SIDIBEMoussa/Application-de-HMM/blob/main/TP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Compléter le code python  à partir du pseudo code fourner

On utilise l'exemple du cours comme référence pratique: 

Supposons que les recherches actuelles indiquent une corrélation entre la taille des cernes de croissance des arbres et la température.

Pour simplification, nous considérons trois tailles différentes de cernes des arbres : petite S , moyenne M et grande L . On ne considère que deux températures annuelles "Chaux : Hot" et " Froid : Cold

Les relations probabiliste sont donnée par :

- La matrice de transition :
$$
A=\left[\begin{array}{ll}
0.7 & 0.3 \\
0.4 & 0.6
\end{array}\right]
$$
- La matrice d'observation des probabilités d'émission :
$$
B=\left[\begin{array}{lll}
0.1 & 0.4 & 0.5 \\
0.7 & 0.2 & 0.1
\end{array}\right]
$$
- la distribution de loi de probabilité de l'état initial:
$$
\pi=\left[\begin{array}{ll}
0.6 & 0.4
\end{array}\right]
$$



In [1]:
import numpy as np

In [29]:
O=np.array([0,1,0,2,1,0,2])                        #Séquence d'observation,0 : S, 1 :M, 2 : L
T=O.shape[0]                                       #Durée de la séquence d'observation
S=['HOT','Cold']                                   #Noms d'États
N=2                                                #Nombre d'états dans le modèle (Froid et Chaud)
M=3                                                #Nombre d'états dans le modèle (Froid et Chaud)


"""1.Initialisation, sélectionnez les valeurs initiales pour les matrices A,B et p"""
p=np.array([0.6,0.4])                             #(à compléter )la distribution de loi de probabilité de l'état initial
A=np.array([[0.7,0.3],[0.4,0.6]])                             #(à compléter )La matrice de transition
B=np.array([[0.1,0.4,0.5],[0.7,0.2,0.1]])                             #(à compléter )La matrice d'observation des probabilités d'émission

print('p=',p)
print('='*20)
print('A=',A)
print('='*20)
print('B=',B)

p= [0.6 0.4]
A= [[0.7 0.3]
 [0.4 0.6]]
B= [[0.1 0.4 0.5]
 [0.7 0.2 0.1]]


In [38]:
def Forward_Backward_Decoding(O,T,N,A,B,p,S):
    """2.Alpha-pass (a-pass)"""
    c=np.zeros((T))                                
    a=np.zeros((T,N))
    """Initialize a[0]"""
    for i in range(0,N):
        a[0][i]= p[i]*B[i][O[0]]       #(à compléter )
        c[0]+=a[0][i]

    """Scale a[0]"""
    c[0]=1/c[0]
    for i in range(0,N):
        a[0][i]=c[0]*a[0][i]

    """Compute a[t]"""
    for t in range(1,T):
        for i in range(0,N):
            for j in range(0,N):
                a[t][i]= a[t][i]+a[t-1][j]*a[j][i]  #(à compléter )
            a[t][i]= a[t][i]*B[i][O[t]] #(à compléter )
            c[t]+=a[t][i]
        #scale a[t][i]
        c[t]=1.0/c[t]
        for i in range(0,N):
            a[t][i]=c[t]*a[t][i]

    """3.Beta pass (b-pass)"""
    b=np.zeros((T,N))
    #Let b[T-1][i]=1, scaled by cT-1
    for i in range(0,N):
        b[T-1][i]=c[T-1]
    #b-pass
    for t in range(T-2,-1,-1):
        for i in range(0,N):
            for j in range(0,N):
                b[t][i]= b[t][i]+A[i][j]*B[j][O[t+1]]*b[t+1][j]  #(à compléter )
            #scale b[t][i]
            b[t][i]=c[t]*b[t][i]

    """4. Compute rt(i,j) and rt(i)"""
    rtij=np.zeros((T,N,N))
    rti=np.zeros((T,N))
    for t in range(0,T-1):
        denom=0.0
        for i in range(0,N):
            for j in range(0,N):
                denom+= a[t][i]*A[i][j]*B[j][O[t+1]]*b[t+1][j]  #(à compléter )
        for i in range(0,N):
            for j in range(0,N):
                rtij[t][i][j]=(a[t][i]*A[i][j]*B[j][O[t+1]]*b[t+1][j])/denom
                rti[t][i]+=rtij[t][i][j]
        
    #Cas particulier pour r[T-1][i]
    denom=0.0
    for i in range(0,N):
        denom+=a[T-1][i]
    for i in range(0,N):
        rti[T-1][i]=a[T-1][i]/denom
    opt=[]
    for t in range(0,T):
        max=0
        state=-1
        for s in range(0,N):
            if rti[t][s]>max:
                max=rti[t][s]
                state=s
        opt.append(S[state])
    print ("Les étapes des états sont "+ ",".join(opt))

In [39]:
Forward_Backward_Decoding(O,T,N,A,B,p,S)

Les étapes des états sont Cold,Cold,Cold,HOT,Cold,Cold,HOT


In [40]:
#On donne ici l'algorithme de viterbi
def VITERBI(O,T,N,A,B,p,S):
    viterbi=np.zeros((N,T)) #Viterbi[s][t] représente le prob. que HMM est dans l'état j après les t premières observations
    opt=[]
    max=0
    state=-1
    for s in range(0,N):
        viterbi[s][0]=p[s]*B[s][O[0]]
        if viterbi[s][0]>max:
            max=viterbi[s][0]
            state=s
    opt.append(S[state])
    for t in range(1,T):
        max_s=0
        state=-1
        for s in range(0,N):
            max=0
            for s2 in range(0,N):
                tmp=viterbi[s2][t-1]*A[s2][s]*B[s][O[t]]
                if tmp>max:
                    max=tmp
            viterbi[s][t]=max
            if viterbi[s][t]>max_s:
                max_s=viterbi[s][t]
                state=s
        opt.append(S[state])
    max=0
    for s in range(0,N):
        tmp=viterbi[s][T-1]
        if tmp>max:
            max=tmp
            state=s
    print ("Les étapes des états sont "+ ",".join(opt) + " avec la plus grande probabilité de %s" %max)

In [41]:
## Résoudre le problème de décodage de HMM : Soit le modèle lamda=(A,B,p) et une séquence d'observation O,
## Trouver une séquence d'états optimale pour le processus de Markkov sous-jacent
## En utilisant deux méthodes :
## 1) Méthode de Viterbi
## 2) Méthode Forward backward

print ("Viterbi method:")
VITERBI(O,T,N,A,B,p,S)
print ("Forward backward method")
Forward_Backward_Decoding(O,T,N,A,B,p,S)

Viterbi method:
Les étapes des états sont Cold,HOT,Cold,HOT,HOT,Cold,HOT avec la plus grande probabilité de 3.3191423999999994e-05
Forward backward method
Les étapes des états sont Cold,Cold,Cold,HOT,Cold,Cold,HOT


In [43]:
#Implementation
#Forward-Backward méthode pour trouver le modèle lamda=(A,B,p) étant donné une séquence d'observation O et les dimensions N et M
import numpy as np
O=np.array([0,1,0,2,1,0,1,1,2,2,1,2,0,1,2,1,2,0])   #Séquence d'observation,0 : S, 1 :M, 2 : L
T=O.shape[0]                                        #Durée de la séquence d'observation
N=2                                                #Nombre d'états dans le modèle (Froid et Chaud)
M=3                                                #Nombre de symboles d'observation
"""1.Initialization, select inital values for matrices A,B and p"""
p= np.array([0.3,0.7])   #(à compléter )
A= np.array([[0.6,0.4],[0.3,0.7]])   #(à compléter )
B= np.array([[0.1,0.4,0.5],[0.6,0.2,0.2]])#(à compléter )
maxIters=100                                        #nombre maximum d'itérations de réestimation
iters=0
OldLogProb=-np.inf
logProb=-20000000.0
while(iters<maxIters and OldLogProb<logProb):
    OldLogProb=logProb
    """2.The alpha-pass (a-pass), compute a(i), which is a N*N matrix"""
    c=np.zeros((T))                                
    a=np.zeros((T,N))
    """Initialize a[0]"""
    for i in range(0,N):
        a[0][i]= p[i]*B[i][O[0]]#(à compléter )
        c[0]+=a[0][i]

    """Scale a[0]"""
    c[0]=1/c[0]
    for i in range(0,N):
        a[0][i]=c[0]*a[0][i]

    """Compute a[t]"""
    for t in range(1,T):
        for i in range(0,N):
            for j in range(0,N):
                a[t][i]= a[t][i]+a[t-1][j]*a[j][i] #(à compléter )
            a[t][i]= a[t][i]*B[i][O[t]] #(à compléter )
            c[t]+=a[t][i]
        #scale a[t][i]
        c[t]=1.0/c[t]
        for i in range(0,N):
            a[t][i]=c[t]*a[t][i]

    """3.The beta pass (b-pass)"""
    b=np.zeros((T,N))
    #Soit b[T-1][i]=1, mis à l'échelle par cT-1
    for i in range(0,N):
        b[T-1][i]=c[T-1]
    #b-pass
    for t in range(T-2,-1,-1):
        for i in range(0,N):
            for j in range(0,N):
                b[t][i]= b[t][i]+A[i][j]*B[j][O[t+1]]*b[t+1][j]#(à compléter )
            #scale b[t][i]
            b[t][i]=c[t]*b[t][i]

    """4. Compute rt(i,j) and rt(i)"""
    rtij=np.zeros((T,N,N))
    rti=np.zeros((T,N))
    for t in range(0,T-1):
        denom=0.0
        for i in range(0,N):
            for j in range(0,N):
                denom+= a[t][i]*A[i][j]*B[j][O[t+1]]*b[t+1][j]#(à compléter )
        for i in range(0,N):
            for j in range(0,N):
                rtij[t][i][j]=(a[t][i]*A[i][j]*B[j][O[t+1]]*b[t+1][j])/denom
                rti[t][i]+=rtij[t][i][j]
        
    #Cas particulier pour r[T-1][i]
    denom=0.0
    for i in range(0,N):
        denom+=a[T-1][i]
    for i in range(0,N):
        rti[T-1][i]=a[T-1][i]/denom

    """5. Ré-estimater A,B and p"""
    # Ré-estimer la partition
    for i in range(0,N):
        p[i]=rti[0][i]                                
    # Ré-estimer A
    for i in range(0,N):
        for j in range(0,N):
            numer=0.0
            denom=0.0
            for t in range(0,T-1):
                numer+=rtij[t][i][j]
                denom+=rti[t][i]
            A[i][j]= numer/denom #(à compléter )

    #Ré-estimer B
    for i in range(0,N):
        for j in range(0,M):
            numer=0.0
            denom=0.0
            for t in range(0,T):
                if(O[t]==j):
                    numer+=rti[t][i]
                denom+=rti[t][i]
            B[i][j]= numer/denom #(à compléter )


    """6. Calculer log[P(O|lamda)]"""
    logProb=0.0
    for i in range(0,T):
        logProb+=np.log(c[i])
    logProb=-logProb
    iters+=1
    print( logProb)
    
print( A)
print( B)
print( p)

-24.540214483564466
-23.3836076739236
-24.532893247702937
[[0.60888005 0.39111995]
 [0.36168169 0.63831831]]
[[0.11447286 0.44014516 0.44538198]
 [0.27847355 0.38867051 0.33285594]]
[0.00354762 0.99645238]
